### Neural Network Lab

Professor: Rick Chakra

TA: Spencer Tilley

The first step is to import pytorch and make sure its the right version!

In [1]:
# Import Pytorch
import torch
print(torch.__version__)

2.1.0+cu118


In [ ]:
#If it wont import because torch isn't installed
#!pip install torch

Lets now import the other packages we need

In [3]:
# Import other packages
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

After importing the remaining packages needed to make a Nural Network we need to set up our dataset to be used by pytorch.

In [4]:
# Difine Dataset
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        self.df = read_csv(path, header=0)
        # store the inputs and outputs
        self.X = self.df.values[:, :-1]
        self.y = self.df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

    # get dataset
    def get_dataset(self):
      return self.df

Next we are going to make a function to use the previous function and split our dataset.

In [5]:
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=47, shuffle=True)
    test_dl = DataLoader(test, batch_size=116, shuffle=False)

    # get df
    df = dataset.get_dataset()
    return df, train_dl, test_dl

Now that we have all of our data functions defined we can call them to prepare our data

In [6]:
#Import and prepare the data
# path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
path = 'ionosphere.csv'
df, train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))

235 116


In [7]:
# view dataset
print(df)

     1  2        3        4        5        6        7        8        9  \
0    1  0  0.99539 -0.05889  0.85243  0.02306  0.83398 -0.37708  1.00000   
1    1  0  1.00000 -0.18829  0.93035 -0.36156 -0.10868 -0.93597  1.00000   
2    1  0  1.00000 -0.03365  1.00000  0.00485  1.00000 -0.12062  0.88965   
3    1  0  1.00000 -0.45161  1.00000  1.00000  0.71216 -1.00000  0.00000   
4    1  0  1.00000 -0.02401  0.94140  0.06531  0.92106 -0.23255  0.77152   
..  .. ..      ...      ...      ...      ...      ...      ...      ...   
346  1  0  0.83508  0.08298  0.73739 -0.14706  0.84349 -0.05567  0.90441   
347  1  0  0.95113  0.00419  0.95183 -0.02723  0.93438 -0.01920  0.94590   
348  1  0  0.94701 -0.00034  0.93207 -0.03227  0.95177 -0.03431  0.95584   
349  1  0  0.90608 -0.01657  0.98122 -0.01989  0.95691 -0.03646  0.85746   
350  1  0  0.84710  0.13533  0.73638 -0.06151  0.87873  0.08260  0.88928   

          10  ...       26       27       28       29       30       31  \
0    0.03760

Now that our data is ready we need to make our model. Unlike the other model types we have used in this class, we need to actually make our model (This can involve a lot of tuning in real life). For this example lets make 3 hidden layers.

In [8]:
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10,8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8,1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X

Now we need to choose how our model will train and improve

In [9]:
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    for epoch in range(150):
      # enumerate mini batches
      for i, (inputs, targets) in enumerate(train_dl):
          # clear the gradients
          optimizer.zero_grad()
          # compute the model output
          yhat = model(inputs)
          # calculate loss
          loss = criterion(yhat, targets)
          # loss assignment
          loss.backward()
          # update model weights
          optimizer.step()

          if i % 3 == 0:
              print(f'Training loss. Epoch: {epoch} Batch: {i} Loss: {loss.item()}')

Now we set up a function to define how to evaluate the model (accuracy)

In [10]:
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

To further show how our model is working lets have the completed model create a prediction on a row of data

In [11]:
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

Now its time to actually run the model and have it use our data!

In [12]:
# define the network
model = MLP(34)
# train the model
train_model(train_dl, model)
# evaluate the model
acc = evaluate_model(test_dl, model)

Training loss. Epoch: 0 Batch: 0 Loss: 0.7229156494140625
Training loss. Epoch: 0 Batch: 3 Loss: 0.7271541953086853
Training loss. Epoch: 1 Batch: 0 Loss: 0.7548994421958923
Training loss. Epoch: 1 Batch: 3 Loss: 0.6991768479347229
Training loss. Epoch: 2 Batch: 0 Loss: 0.6560651063919067
Training loss. Epoch: 2 Batch: 3 Loss: 0.6178153157234192
Training loss. Epoch: 3 Batch: 0 Loss: 0.6290187239646912
Training loss. Epoch: 3 Batch: 3 Loss: 0.4911806583404541
Training loss. Epoch: 4 Batch: 0 Loss: 0.5756160020828247
Training loss. Epoch: 4 Batch: 3 Loss: 0.544249951839447
Training loss. Epoch: 5 Batch: 0 Loss: 0.5162132382392883
Training loss. Epoch: 5 Batch: 3 Loss: 0.5077219009399414
Training loss. Epoch: 6 Batch: 0 Loss: 0.4451996386051178
Training loss. Epoch: 6 Batch: 3 Loss: 0.5680035948753357
Training loss. Epoch: 7 Batch: 0 Loss: 0.5897856950759888
Training loss. Epoch: 7 Batch: 3 Loss: 0.5630333423614502
Training loss. Epoch: 8 Batch: 0 Loss: 0.4514332115650177
Training loss. 

In [13]:
# print final model accuracy
print('Final Model Accuracy: %.3f' % acc)

Final Model Accuracy: 0.905


In [15]:
# make a single prediction (expect class=1)
row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))

Predicted: 0.995 (class=1)


One of the reasons that we use functions so much in this section is it allows for an easier time making different models and comparing how they preform!